In [2]:
import torch


In [4]:
x = torch.tensor([[2,1],[5,6],[3,7],[12,0]])
y = torch.tensor([0,1,0,1],dtype=torch.float)

In [5]:
edge_index = torch.tensor([[0,1,2,0,3],
                         [1,0,1,3,2]],dtype=torch.long)

In [6]:
from torch_geometric.data import Data

data = Data(x=x,y=y,edge_index=edge_index)

In [7]:
data

Data(edge_index=[2, 5], x=[4, 2], y=[4])

In [11]:
data['x']

tensor([[ 2,  1],
        [ 5,  6],
        [ 3,  7],
        [12,  0]])

In [12]:
data['y']

tensor([0., 1., 0., 1.])

In [13]:
data['edge_index']

tensor([[0, 1, 2, 0, 3],
        [1, 0, 1, 3, 2]])

In [14]:
from torch_geometric.data import InMemoryDataset

In [17]:
print(data.keys)

['x', 'edge_index', 'y']


In [18]:
data.num_nodes

4

In [19]:
data.num_edges

5

In [20]:
data.num_node_features

2

In [22]:
data.contains_isolated_nodes()

False

In [23]:
data.contains_self_loops()

False

In [24]:
data.is_directed()

True

In [25]:
from torch_geometric.datasets import TUDataset

In [44]:
dataset = TUDataset('ENZYMES',name='ENZYMES')

In [27]:
len(dataset)

600

In [28]:
dataset.num_classes

6

In [29]:
dataset.num_features

3

In [30]:
data = dataset[0]

In [31]:
data

Data(edge_index=[2, 168], x=[37, 3], y=[1])

In [33]:
data.is_undirected()

True

In [34]:
from torch_geometric.datasets import Planetoid

In [35]:
dataset = Planetoid('Cora',name='Cora')

Processing...
Done!


In [36]:
len(dataset)

1

In [37]:
dataset.num_classes

7

In [38]:
dataset.num_node_features

1433

In [39]:
data = dataset[0]
data

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])

In [40]:
data.is_undirected()

True

In [41]:
data.train_mask.sum().item()

140

In [42]:
data.val_mask.sum().item()

500

In [43]:
data.test_mask.sum().item()

1000

In [45]:
from torch_geometric.data import DataLoader

loader = DataLoader(dataset,batch_size=32,shuffle=True)


In [46]:
for batch in loader:
    print(batch)

Batch(batch=[1121], edge_index=[2, 4358], x=[1121, 3], y=[32])
Batch(batch=[975], edge_index=[2, 3780], x=[975, 3], y=[32])
Batch(batch=[985], edge_index=[2, 3820], x=[985, 3], y=[32])
Batch(batch=[1018], edge_index=[2, 3974], x=[1018, 3], y=[32])
Batch(batch=[982], edge_index=[2, 3794], x=[982, 3], y=[32])
Batch(batch=[1301], edge_index=[2, 4872], x=[1301, 3], y=[32])
Batch(batch=[985], edge_index=[2, 3714], x=[985, 3], y=[32])
Batch(batch=[984], edge_index=[2, 3830], x=[984, 3], y=[32])
Batch(batch=[901], edge_index=[2, 3496], x=[901, 3], y=[32])
Batch(batch=[1009], edge_index=[2, 3956], x=[1009, 3], y=[32])
Batch(batch=[1136], edge_index=[2, 4134], x=[1136, 3], y=[32])
Batch(batch=[1066], edge_index=[2, 4114], x=[1066, 3], y=[32])
Batch(batch=[1006], edge_index=[2, 3800], x=[1006, 3], y=[32])
Batch(batch=[1070], edge_index=[2, 4018], x=[1070, 3], y=[32])
Batch(batch=[891], edge_index=[2, 3524], x=[891, 3], y=[32])
Batch(batch=[988], edge_index=[2, 3748], x=[988, 3], y=[32])
Batch(ba

In [47]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid('Planetoid',name='Cora')


Processing...
Done!


In [57]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class net(torch.nn.Module):
    def __init__(self):
        super(net,self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features,16)
        self.conv2 = GCNConv(16,dataset.num_classes)
        
    def forward(self,data):
        x,edge_index = data.x,data.edge_index
        
        x = self.conv1(x,edge_index)
        x = F.relu(x)
        
        x = F.dropout(x,training=self.training)
        x = self.conv2(x,edge_index)
        
        return F.log_softmax(x,dim=1)

In [58]:
device = torch.device('cpu')

model = net().to(device)

data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=0.01,weight_decay=5e-4)
model.train()

for epoch in range(100):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask],data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [59]:
model.eval()
_,pred = model(data).max(dim=1)

correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct/data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.7890
